# Introduction to Distributed Computing (10 mins)

[![Slack Status](https://img.shields.io/badge/slack-join_chat-white.svg?logo=slack&style=social)](http://slack.fugue.ai)

Before diving into code, let's first take a look at the the current tooling out there, and the use cases that demand distributed computing.

In this section, we explore:

* when do I need distributed computing?
* is big data still a thing?
* what does the big data ecosystem look like?
* what are the issues with current frameworks?

In [ ]:
_=!mamba install -y openjdk
_=!pip install -r ../requirements.txt

## When We Need Distributed Computing

pandas is great for small datasets, but unfortunately does not scale well large datasets. The primary reason is that pandas is single core, and does not take advantage of all available compute resources. A lot of operations also generate [intermediate copies](https://pandas.pydata.org/pandas-docs/stable/user_guide/scale.html#scaling-to-large-datasets) of data, utilizing more memory than necessary. To effectively handle data with pandas, users preferably need to have [5x to 10x times](https://wesmckinney.com/blog/apache-arrow-pandas-internals/) as much RAM as the size of the dataset.

Spark and Dask allow us to split compute jobs across multiple machines. They also can handle datasets that don’t fit into memory by [spilling data](http://distributed.dask.org/en/latest/worker.html#spill-data-to-disk) over to disk in some cases. This feature should be used sparingly though.

The `dask-ml` documentation has a good diagram of the types of problems that call for distributed computing. The general advice is to only introduce distributed computng when you need it because of the additional complexity that comes with building and maintaing such solutions (more on this later).

<img src="https://ml.dask.org/_images/dimensions_of_scale.svg" align="left" width="700"/>

## Is Big Data Dead?

![img](../images/polars_duckdb.png)

There was an article by [MotherDuck](https://motherduck.com/blog/big-data-is-dead/) saying big data is dead. There are a lot of reasons, but the one most relevant to us is that tools like [Polars](https://github.com/pola-rs/polars) and [DuckDB](https://github.com/duckdb/duckdb) allow users to process significantly larger amounts of data on a single machine. It is true that tooling on the local ecosystem is better, but as shown in the graph above, speed can also be a concern. We can still use distributed computing for use cases like training multiple machine learning models over a cluster.

Second, there are still large data sources as mentioned by the [rebuttal blog by Ponder](https://ponder.io/big-data-is-dead-long-live-big-data/). Transactional and event-based data can generate a lot of data, and are often underutilized because they are left in cold storage. To utilize this data effectively, we need to leverage big data tools. We don't need big data tooling for all projects, but we do need to leverage them in certain use cases.

## Distributed Computing Architecture

There is an image in the Dask repo [issues](https://github.com/dask/dask/issues/4471) that clearly illustrates the distributed computing paradigm. In general, there is a client is responsible for interacting with the scheduler that takes care of the orchestration and final data collection. 

All Spark, Dask, and Ray have local modes also where they use the cores available on the local machine. This means we can still take advantage of the additional processing without having a cluster available. 

In the diagram below, note how data actually lives on a physical machine

<img src="https://user-images.githubusercontent.com/11656932/62263986-bbba2f00-b3e3-11e9-9b5c-8446ba4efcf9.png" align="left" width="700"/>

## Introductions to Partitions

In order to understand partitions, we can look at this image showing the way Dask scales Pandas. Each partition is a Pandas DataFrame. A Dask DataFrame is the collection of all of the Pandas DataFrames. Operations are done on each partition, and then aggregated back.

<img src="https://docs.dask.org/en/latest/_images/dask-dataframe.svg" align="left" width="400"/>

## Distributed Computing Ecosystem 

![img](../images/spark_dask_ray.png)

## Pain Points of Distributed Computing

* Iteration is harder (and costlier)
* Code can be harder to test
* Code is locked in to a framework, and can look very different
* Requires expertise to maintain

## Examples of Differing Syntax

Let's see an example where we do a `groupby.median()`.

In [27]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'group': np.random.choice(["A","B"], 50),
                   'col1': np.random.randint(1, 100, 50)})
df.head()

,group,col1
0,A,57
1,B,88
2,B,90
3,B,53
4,B,50


**Pandas**

In [29]:
def pandas_median(df):
    return df.groupby("group").median()

pandas_median(df).reset_index()

,group,col1
0,A,55.5
1,B,47.5


**PySpark**

In [33]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

sdf = spark.createDataFrame(df)

In [36]:
import pyspark.sql.functions as F

def spark_median(sdf):
    med_func = F.expr('percentile_approx(col1, 0.5, 10)').alias('median')
    return sdf.groupBy('group').agg(med_func)

spark_median(sdf).show()

+-----+------+
|group|median|
+-----+------+
|    B|    43|
|    A|    45|
+-----+------+



In the next example, we look at mapping over a dictionary

In [38]:
mapper = {"A": "Apple", "B": "Banana"}
def map_letter_to_food(df):
    return df.assign(group=df['group'].replace(mapper))

map_letter_to_food(df).head()

,group,col1
0,Apple,57
1,Banana,88
2,Banana,90
3,Banana,53
4,Banana,50


In [43]:
from itertools import chain
from pyspark.sql.functions import create_map, lit

def map_letter_to_food_spark(sdf):
    mapping = create_map([lit(x) for x in chain(*mapper.items())])
    return sdf.withColumn("group", mapping[sdf['group']])

map_letter_to_food_spark(sdf).show(5)

+------+----+
| group|col1|
+------+----+
| Apple|  57|
|Banana|  88|
|Banana|  90|
|Banana|  53|
|Banana|  50|
+------+----+
only showing top 5 rows



## Why is this Problematic

1. Our code becomes framework dependent. We can't reuse the same Spark logic for Pandas-sized projects.
2. Even if you have the bandwidth to move everything, there can be different behavior (we'll see more later)
3. Code becomes a lot harder to maintain (needs expertise). StackOverflow shows 5x for Pandas users than Spark users.

So what if we could have a layer that lets us toggle between Pandas and Spark (or Dask, and Ray)?